# Kinematic Trajectory Optimization
## Evaluating throw and catch points

In [ ]:
import time

import numpy as np
from pydrake.all import (AddMultibodyPlantSceneGraph, BsplineTrajectory,
                         DiagramBuilder, KinematicTrajectoryOptimization,
                         MeshcatVisualizer, MeshcatVisualizerParams,
                         MinimumDistanceConstraint, Parser, PositionConstraint,
                         Rgba, RigidTransform, Role, Solve, Sphere,
                         StartMeshcat, JacobianWrtVariable, RollPitchYaw,
                         JointSliders, RotationMatrix,
                         InverseKinematics,
                         LeafSystem, AbstractValue,
                         Simulator
                        )

from manipulation import running_as_notebook
from manipulation.meshcat_utils import (PublishPositionTrajectory,
                                        MeshcatPoseSliders,
                                        WsgButton)
from manipulation.scenarios import AddIiwaDifferentialIK, MakeManipulationStation, AddIiwa, AddPlanarIiwa, AddShape, AddWsg, AddMultibodyTriad 
from manipulation.utils import AddPackagePaths, FindResource

from IPython.display import clear_output


In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://68a898be-bd4e-4046-916a-6293aadf6c5d.deepnoteproject.com/7000/
Installing NginX server for MeshCat on Deepnote...


To visualize throw and catch points

In [ ]:
squash_ball_radius = 0.02
p_GB_G = [0, 0.07, 0]
def add_target_objects(X_WThrow, X_WCatch):
    p_ThrowB_W = X_WThrow.rotation() @ p_GB_G
    X_WThrowB_W = RigidTransform(X_WThrow.rotation(), X_WThrow.translation() +  p_ThrowB_W)
    p_CatchB_W = X_WCatch.rotation() @ p_GB_G
    X_WCatchB_W = RigidTransform(X_WCatch.rotation(), X_WCatch.translation() +  p_CatchB_W)
    meshcat.SetObject("throw", Sphere(squash_ball_radius), rgba=Rgba(.9, .1, .1, 1))
    meshcat.SetTransform("throw", X_WThrowB_W)
    meshcat.SetObject("catch", Sphere(squash_ball_radius), rgba=Rgba(.1, .9, .1, 1))
    meshcat.SetTransform("catch", X_WCatchB_W)

## Forward Kinematics approach
To finding comfortable throw and catch positions

In [ ]:
import numpy as np
from IPython.display import clear_output, display
from pydrake.all import (AbstractValue, AddMultibodyPlantSceneGraph,
                         DiagramBuilder, JointSliders, LeafSystem,
                         MeshcatVisualizer, Parser, RigidTransform,
                         RollPitchYaw, StartMeshcat)

from manipulation import FindResource, running_as_notebook
from manipulation.scenarios import AddMultibodyTriad, AddPackagePaths

meshcat = StartMeshcat()

In [ ]:
def gripper_forward_kinematics_example():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0)
    parser = Parser(plant)
    AddPackagePaths(parser)
    parser.AddAllModelsFromFile(FindResource("models/iiwa_and_wsg.dmd.yaml"))
    plant.Finalize()

    # Draw the frames
    for body_name in ["iiwa_link_1", "iiwa_link_2", "iiwa_link_3", "iiwa_link_4", "iiwa_link_5", "iiwa_link_6", "iiwa_link_7", "body"]:
        AddMultibodyTriad(plant.GetFrameByName(body_name), scene_graph)

    meshcat.Delete()
    meshcat.DeleteAddedControls()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph.get_query_output_port(), meshcat)

    wsg = plant.GetModelInstanceByName("wsg")
    gripper = plant.GetBodyByName("body", wsg)

    X_WResult = None
    my_context = None
    class PrintPose(LeafSystem):
        def __init__(self, body_index):
            LeafSystem.__init__(self)
            self._body_index = body_index
            self.DeclareAbstractInputPort("body_poses",
                                        AbstractValue.Make([RigidTransform()]))
            self.DeclareForcedPublishEvent(self.Publish)

        def Publish(self, context):
            pose = self.get_input_port().Eval(context)[self._body_index]
            clear_output(wait=True)
            print("gripper position (m): " + np.array2string(
                pose.translation(), formatter={
                    'float': lambda x: "{:3.2f}".format(x)}))
            print("gripper roll-pitch-yaw (rad):" + np.array2string(
                RollPitchYaw(pose.rotation()).vector(),
                            formatter={'float': lambda x: "{:3.2f}".format(x)}))
            nonlocal X_WResult
            X_WResult = pose

    print_pose = builder.AddSystem(PrintPose(gripper.index()))
    builder.Connect(plant.get_body_poses_output_port(),
                    print_pose.get_input_port())

    default_interactive_timeout = None if running_as_notebook else 1.0
    sliders = builder.AddSystem(JointSliders(meshcat, plant))
    diagram = builder.Build()
    

    sliders.Run(diagram, default_interactive_timeout)
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    meshcat.DeleteAddedControls()
    return plant.GetPositions(plant_context), X_WResult

Set throw point via forward kinematics teleop

In [ ]:
q_Throw, X_WThrow = gripper_forward_kinematics_example()
print(f"q_Throw: {q_Throw}")
print(f"X_WThrow: {X_WThrow}")

gripper position (m): [0.40 -0.24 0.68]
gripper roll-pitch-yaw (rad):[-1.81 0.00 1.03]
q_Throw: [-1.57  0.1   0.   -1.2   0.    1.6   0.    0.    0.  ]
X_WThrow: RigidTransform(
  R=RotationMatrix([
    [0.5141359916531135, 0.20520622667730418, -0.8327992474898938],
    [0.8577086813638249, -0.1230066911277729, 0.49920453909287205],
    [-1.7156031564295772e-18, -0.970958165149591, -0.23924932921398243],
  ]),
  p=[0.39936370505596896, -0.2393904352264749, 0.6793215789060887],
)


Manually set the throw and catch points

In [ ]:
q_Throw = np.array([-0.54, 0.58, 0, -1.79, 0, -0.79, 0])
p_WThrow = [0.57, -0.34, 0.39]
R_WThrow = RollPitchYaw([-np.pi, 0.00, 1.03])

q_Catch = np.array([-0.74, 0.58, 0, -1.79, 0, -0.79, 0])
p_WCatch = [0.49, -0.45, 0.39]
R_WCatch = RollPitchYaw([-np.pi, 0.00, 0.83])

X_WThrow = RigidTransform(R_WThrow, p_WThrow)
X_WCatch = RigidTransform(R_WCatch, p_WCatch)

# WSG gripper
opened = np.array([0.035]) #[0.035]
closed = np.array([0.02])

# Differential IK Controller

Use diffIK controller to get other joint configurations around the key points, navigated to using spatial translation

In [ ]:
def teleop_3d():
    builder = DiagramBuilder()

    time_step = 0.001
    station = builder.AddSystem(
        MakeManipulationStation(filename="./models/one_arm_juggling.dmd.yaml", time_step=time_step))
    plant = station.GetSubsystemByName("plant")
    controller_plant = station.GetSubsystemByName(
        "iiwa_controller").get_multibody_plant_for_control()

    # Add a meshcat visualizer.
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat)
    meshcat.ResetRenderMode()
    meshcat.DeleteAddedControls()

    # Set up differential inverse kinematics.
    differential_ik = AddIiwaDifferentialIK(
        builder,
        controller_plant,
        frame=controller_plant.GetFrameByName("iiwa_link_7"))
    builder.Connect(differential_ik.get_output_port(),
                    station.GetInputPort("iiwa_position"))
    builder.Connect(station.GetOutputPort("iiwa_state_estimated"),
                    differential_ik.GetInputPort("robot_state"))

    # Set up teleop widgets.
    teleop = builder.AddSystem(
        MeshcatPoseSliders(
            meshcat,
            min_range=MeshcatPoseSliders.MinRange(roll=0,
                                                  pitch=-0.5,
                                                  yaw=-np.pi,
                                                  x=-0.6,
                                                  y=-0.8,
                                                  z=0.0),
            max_range=MeshcatPoseSliders.MaxRange(roll=2 * np.pi,
                                                  pitch=np.pi,
                                                  yaw=np.pi,
                                                  x=0.8,
                                                  y=0.3,
                                                  z=1.1),
            body_index=plant.GetBodyByName("iiwa_link_7").index()))
    builder.Connect(teleop.get_output_port(0),
                    differential_ik.get_input_port(0))
    builder.Connect(station.GetOutputPort("body_poses"),
                    teleop.GetInputPort("body_poses"))
    wsg_teleop = builder.AddSystem(WsgButton(meshcat))
    builder.Connect(wsg_teleop.get_output_port(0),
                    station.GetInputPort("wsg_position"))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    plant_context = plant.GetMyMutableContextFromRoot(context)
    q0 = plant.GetPositions(plant_context)
    non_iiwa_q0 = q0[7:]
    gap = closed[0]
    non_iiwa_q0[0]=-gap/2
    non_iiwa_q0[1]=gap/2
    plant.SetPositions(plant_context, np.concatenate((q_Catch, non_iiwa_q0)))

    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)

        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
            print(plant.get_positi)
        meshcat.DeleteButton("Stop Simulation")

    else:
        simulator.AdvanceTo(0.1)


teleop_3d()

Keyboard Controls:
roll : KeyQ / KeyE
pitch : KeyW / KeyS
yaw : KeyA / KeyD
x : KeyJ / KeyL
y : KeyI / KeyK
z : KeyO / KeyU
Press Space to open/close the gripper
Press Escape to stop the simulation


## Inverse Kinematics Approach

#### Start with the throw and catch spatial positions in XYZ

In [ ]:
a = 0.4
b = 0.6
z = 0.2
p_WThrow = [b,-a,z]
R_WThrow = RollPitchYaw([-np.pi, 0, np.arctan(a/b)])
print(R_WThrow)
p_WCatch = [a,-b,z]
R_WCatch = RollPitchYaw([-np.pi, 0, np.arctan(b/a)])
print(R_WCatch)
X_WThrow = RigidTransform(R_WThrow, p_WThrow)
X_WCatch = RigidTransform(R_WCatch, p_WCatch)

In [ ]:
add_target_objects(X_WThrow, X_WCatch)

### Use interactive inverse kinematics to get joint angles for throw and catch points

In [ ]:

def get_joints_inverse_kinematics(X_WTarget, use_full_pose = True):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    iiwa = AddIiwa(plant, "with_box_collision")
    wsg = AddWsg(plant, iiwa, welded=True)
    plant.Finalize()

    meshcat.DeleteAddedControls()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, 
        scene_graph, 
        meshcat,
        MeshcatVisualizerParams(prefix="Spatial"))
    joint_sliders = builder.AddSystem(JointSliders(meshcat, plant))

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    q0 = plant.GetPositions(plant_context)
    gripper_frame = plant.GetFrameByName("body", wsg)
    q_result = None
    def my_callback(context, pose):
        nonlocal q_result
        ik = InverseKinematics(plant, plant_context)
        ik.AddPositionConstraint(
            gripper_frame, p_GO, plant.world_frame(),
            pose.translation(), pose.translation())
        ik.AddOrientationConstraint(
            gripper_frame, RotationMatrix(), plant.world_frame(),
            pose.rotation(), 0.0)
        ik.AddMinimumDistanceConstraint(0.001, 0.1)
        prog = ik.get_mutable_prog()
        q = ik.q()
        prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
        prog.SetInitialGuess(q, q0)
        result = Solve(ik.prog())
        # clear_output(wait=True)
        if result.is_success():
            print("IK success")
            joint_sliders.SetPositions(result.GetSolution())
            print(f"Joint positions: {result.GetSolution()}")
            q_result = result.GetSolution()
        else:
            print("IK failure")

    sliders = MeshcatPoseSliders(meshcat)
    sliders.SetPose(plant.EvalBodyPoseInWorld(
        plant_context, plant.GetBodyByName("body", wsg)))
    if use_full_pose:
        sliders.SetPose(X_WTarget)
    else:
        sliders.SetXyz(X_WTarget.translation())
    sliders.Run(visualizer, context, my_callback)
    return q_result


In [ ]:
q_Throw = get_joints_inverse_kinematics(X_WThrow, use_full_pose=True)
print(q_Throw)

In [ ]:
q_Catch = get_joints_inverse_kinematics(X_WCatch, use_full_pose=False)
print(q_Catch)

## Calculate spatial velocities at throw and catch points

In [ ]:
def calculate_ball_vels(p1, p2, height):
    # p1: (x, y, z), ndarray
    # p2: (x, y, z), ndarray
    # height, pos real number

    g = 9.8

    t = np.sqrt(2 * height / g)

    vx = (p2[0] - p1[0]) / (2 * t)
    vy = (p2[1] - p1[1]) / (2 * t)
    vz = t * g

    throw_vel = np.array([vx, vy, vz])
    catch_vel = np.array([vx, vy, -vz])
    total_duration =  2 * t
    return (throw_vel, catch_vel, total_duration)

In [ ]:

max_height = 0.5 # Max height ball will reach in meters
V_Throw, V_Catch, t_duration = calculate_ball_vels(X_WThrow.translation(), X_WCatch.translation(), max_height)
print(f"V_Throw {V_Throw}")
print(f"V_Catch {V_Catch}")
print(f"t_duration {t_duration}")

V_Throw [-0.12521981 -0.17217723  3.13049517]
V_Catch [-0.12521981 -0.17217723 -3.13049517]
t_duration 0.6388765649999398


In [ ]:
# Override to reduce velocities
# not necessary
# V_Throw[2] = 1
# V_Catch[2] = -1

In [ ]:
def SpatialVelToJointVelConverter():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    iiwa = AddIiwa(plant)
    wsg = AddWsg(plant, iiwa, welded=True, sphere=True)
    gripper_frame = plant.GetFrameByName("body", wsg)
    plant.Finalize()

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    def convert_spatial_vel_to_joint_vel(q, V_Gdesired):
        plant.SetPositions(plant_context, q)
        diagram.Publish(context)

        clear_output(wait=True)
        J_G = plant.CalcJacobianTranslationalVelocity(plant_context,
                                                      JacobianWrtVariable.kQDot,
                                                      gripper_frame, [0, 0, 0],
                                                      plant.world_frame(),
                                                      plant.world_frame())
        # print("J_G = ")
        # print(
        #     np.array2string(J_G,
                            # formatter={'float': lambda x: "{:5.2f}".format(x)}))
        # print(np.shape(J_G))
        # print(np.shape())
        V_Gdesired = V_Gdesired.reshape(3,1)
        v = np.linalg.pinv(J_G).dot(V_Gdesired)
        # print("Joint velocities")
        # print(np.array2string(v,
        #                     formatter={'float': lambda x: "{:5.3f}".format(x)}))
        return v

    return convert_spatial_vel_to_joint_vel

converter = SpatialVelToJointVelConverter()

In [ ]:
v_Throw = converter(q_Throw, V_Throw)
print(np.shape(v_Throw))
print(np.array2string(v_Throw, formatter={'float': lambda x: "{:5.3f}".format(x)}))
# v_Throw = np.array([[-0.253],[-4.044],[-0.202],[0.597],[0.046],[-0.943],[-0.000]])

(7, 1)
[[-0.253]
 [-4.044]
 [-0.202]
 [0.597]
 [0.046]
 [-0.943]
 [-0.000]]


In [ ]:
v_Catch = converter(q_Catch,V_Catch)
print(np.array2string(v_Catch, formatter={'float': lambda x: "{:5.3f}".format(x)}))
# v_Catch = np.array([[-0.250],[4.103],[-0.200],[-0.505],[0.045],[0.953],[0.000]])

[[-0.250]
 [4.103]
 [-0.200]
 [-0.505]
 [0.045]
 [0.953]
 [0.000]]


In [ ]:
def trajopt_pv_pair_demo():
    meshcat.Delete()
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

    parser = Parser(plant)
    AddPackagePaths(parser)
    iiwa = AddIiwa(plant)
    wsg = AddWsg(plant, iiwa, welded=True, sphere=True)
    
    add_target_objects(X_WThrow, X_WCatch)

    t_start = 0.0 # 0.5
    num_control_points = 11

    plant.Finalize()

    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat,
        MeshcatVisualizerParams(role=Role.kIllustration))
    collision_visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat,
        MeshcatVisualizerParams(prefix="collision", role=Role.kProximity))
    meshcat.SetProperty("collision", "visible", False)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    # print(q_Throw)
    plant.SetPositions(plant_context, q_Throw)
    q0 = plant.GetPositions(plant_context)
    # print(q0)
    
    gripper_frame = plant.GetFrameByName("body", wsg)
    trajopt = KinematicTrajectoryOptimization(plant.num_positions(), num_control_points)
    prog = trajopt.get_mutable_prog()
    
    trajopt.AddPathLengthCost(2)
    # Add cost for acceleration?

    trajopt.AddPositionBounds(plant.GetPositionLowerLimits(),
                              plant.GetPositionUpperLimits())
    trajopt.AddVelocityBounds(plant.GetVelocityLowerLimits(),
                              plant.GetVelocityUpperLimits())

    # trajopt.AddDurationBounds(t_start, t_start + 2 * t_duration) # Assume t_duration each way
    trajopt.AddDurationConstraint(t_start + 2 * t_duration, t_start + 2 * t_duration)
    # trajopt.AddEqualTimeIntervalConstraints()

    # Constrain positions in spatial positions
    # start constraint
    # start_constraint = PositionConstraint(plant, plant.world_frame(),
    #                                       X_WThrow.translation(),
    #                                       X_WThrow.translation(), gripper_frame,
    #                                       [0, 0.1, 0], plant_context)
    # trajopt.AddPathConstraint(start_constraint, 0)
    prog.AddQuadraticErrorCost(np.eye(7), q0,
                               trajopt.path_control_points()[:, 0])
    
    # goal constraint
    # goal_constraint = PositionConstraint(plant, plant.world_frame(),
    #                                      X_WCatch.translation(),
    #                                      X_WCatch.translation(), gripper_frame,
    #                                      [0, 0.1, 0], plant_context)
    
    # trajopt.AddPathConstraint(goal_constraint, 0.5)
    prog.AddQuadraticErrorCost(np.eye(7), q_Catch,
                               trajopt.path_control_points()[:, np.round(num_control_points/2).astype(int)])

    # return to start
    # trajopt.AddPathConstraint(start_constraint, 1)
    prog.AddQuadraticErrorCost(np.eye(7), q0,
                               trajopt.path_control_points()[:, -1])
    
    # constrain positions in joint angles
    trajopt.AddFixedPathPositionConstraint(q_Throw, 0)
    trajopt.AddFixedPathPositionConstraint(q_Catch, 0.5)
    trajopt.AddFixedPathPositionConstraint(q_Throw, 1)

    # print(v_Throw)
    trajopt.AddFixedPathVelocityConstraint(v_Throw, 0)
    trajopt.AddFixedPathVelocityConstraint(v_Catch, 0.5)
    trajopt.AddFixedPathVelocityConstraint(v_Throw, 1)

    # Solve once without the collisions and set that as the initial guess for
    # the version with collisions.
    result = Solve(prog)
    if not result.is_success():
        print("Trajectory optimization failed, even without collisions!")
        print(result.get_solver_id().name())
    
    # BsplineTrajectory()
    # trajopt.SetPathInitialGuess()
    traj = trajopt.ReconstructPath(result)
    print(f"kin traj opt initial t:{traj.start_time()} q: {traj.InitialValue()}")
    print(f"kin traj opt final t:{traj.end_time()} q: {traj.FinalValue()}")

    trajopt.SetPathInitialGuess(trajopt.ReconstructPath(result))

    PublishPositionTrajectory(trajopt.ReconstructPath(result), context, plant,
                              visualizer)
    collision_visualizer.Publish(
        collision_visualizer.GetMyContextFromRoot(context))

    return
    # collision constraints
    collision_constraint = MinimumDistanceConstraint(plant, 0.001,
                                                     plant_context, None, 0.01)
    evaluate_at_s = np.linspace(0, 1, 25)
    for s in evaluate_at_s:
        trajopt.AddPathConstraint(collision_constraint, s)

    result = Solve(prog)
    if not result.is_success():
        print("Trajectory optimization failed")
        print(result.get_solver_id().name())

    PublishPositionTrajectory(trajopt.ReconstructPath(result), context, plant,
                              visualizer)
    collision_visualizer.Publish(
        collision_visualizer.GetMyContextFromRoot(context))
    
    

trajopt_pv_pair_demo()


AttributeError: 'pydrake.systems.trajectory_optimization.KinematicT' object has no attribute 'AddDurationConstraint'

In [ ]:
meshcat.Delete()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=68a898be-bd4e-4046-916a-6293aadf6c5d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>